# Mosaico 1985

Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Mosaicos Patagonia

2021-08-05 13:07:09 

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [2]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Colors Bars

In [4]:
import geemap.colormaps as cm

In [5]:
# cm.palettes.terrain

In [6]:
color_ndvi = cm.palettes.ndvi

In [7]:
color_ndwi = cm.palettes.ndwi

In [8]:
color_terrain = cm.palettes.terrain

In [9]:
color_glaciar = cm.palettes.PuBuGn_r

In [10]:
color_nubes = cm.palettes.inferno

In [11]:
# cm.list_colormaps()

In [12]:
# cm.plot_colormaps(width=12, height=0.4)

## Funciones

In [13]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [14]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('B11')
    return image.addBands(ndvi)

In [15]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [16]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [17]:
def addSAVI(image):
    savi = image.expression(
        '((b4 - b3)/(b4 + b3 + 0.5))*1.5', {
            'b3':image.select('B3'),
            'b4':image.select('B4')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

In [18]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B2','B5']).rename('NDSI')
    return image.addBands(ndsi)

In [19]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMM'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [20]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [21]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [22]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [23]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 1985

In [24]:
start_date = '1984-10-01'
end_date = '1985-03-31'

l5_1985 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [25]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [26]:
vis_nieve = {
  'bands': ['B5', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [27]:
coleccion1985 = l5_1985.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [28]:
QM_NDVI_1985 = coleccion1985.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1985 = coleccion1985.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1985 = coleccion1985.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1985 = coleccion1985.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1985 = coleccion1985.qualityMosaic('NDSI') # Normalized difference snow index

In [29]:
Map.addLayer(QM_NDVI_1985, vizParams, 'QM de NDVI 1985')
Map.addLayer(QM_NDWI_1985, vizParams, 'QM de NDWI 1985')
Map.addLayer(QM_SAVI_1985, vizParams, 'QM de SAVI 1985')
Map.addLayer(QM_NDGI_1985, vis_nieve, 'QM de NDGI 1985')
Map.addLayer(QM_NDSI_1985, vis_nieve, 'QM de NDSI 1985')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [30]:
c1985_sn = coleccion1985.map(Cloud20)
NDVI_1985_sn = c1985_sn.qualityMosaic('B11')
NDWI_1985_sn = c1985_sn.qualityMosaic('NDWI')
SAVI_1985_sn = c1985_sn.qualityMosaic('SAVI')
NDGI_1985_sn = c1985_sn.qualityMosaic('NDGI')
NDSI_1985_sn = c1985_sn.qualityMosaic('NDSI')

In [31]:
Map2.addLayer(NDVI_1985_sn, vizParams, 'QMsn de NDVI 1985')
Map2.addLayer(NDWI_1985_sn, vizParams, 'QMsn de NDWI 1985')
Map2.addLayer(SAVI_1985_sn, vizParams, 'QMsn de SAVI 1985')
Map2.addLayer(NDGI_1985_sn, vis_nieve, 'QMsn de NDGI 1985')
Map2.addLayer(NDSI_1985_sn, vis_nieve, 'QMsn de NDSI 1985')
# Map2

### 1986

In [32]:
start_date = '1985-10-01'
end_date = '1986-03-31'

l5_1986 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [33]:
coleccion1986 = l5_1986.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [34]:
QM_NDVI_1986 = coleccion1986.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1986 = coleccion1986.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1986 = coleccion1986.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1986 = coleccion1986.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1986 = coleccion1986.qualityMosaic('NDSI') # Normalized difference snow index

In [35]:
Map.addLayer(QM_NDVI_1986, vizParams, 'QM de NDVI 1986')
Map.addLayer(QM_NDWI_1986, vizParams, 'QM de NDWI 1986')
Map.addLayer(QM_SAVI_1986, vizParams, 'QM de SAVI 1986')
Map.addLayer(QM_NDGI_1986, vis_nieve, 'QM de NDGI 1986')
Map.addLayer(QM_NDSI_1986, vis_nieve, 'QM de NDSI 1986')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [36]:
c1986_sn = coleccion1986.map(Cloud20)
NDVI_1986_sn = c1986_sn.qualityMosaic('B11')
NDWI_1986_sn = c1986_sn.qualityMosaic('NDWI')
SAVI_1986_sn = c1986_sn.qualityMosaic('SAVI')
NDGI_1986_sn = c1986_sn.qualityMosaic('NDGI')
NDSI_1986_sn = c1986_sn.qualityMosaic('NDSI')

In [37]:
Map2.addLayer(NDVI_1986_sn, vizParams, 'QMsn de NDVI 1986')
Map2.addLayer(NDWI_1986_sn, vizParams, 'QMsn de NDWI 1986')
Map2.addLayer(SAVI_1986_sn, vizParams, 'QMsn de SAVI 1986')
Map2.addLayer(NDGI_1986_sn, vis_nieve, 'QMsn de NDGI 1986')
Map2.addLayer(NDSI_1986_sn, vis_nieve, 'QMsn de NDSI 1986')
# Map2

### 1987

In [38]:
start_date = '1986-10-01'
end_date = '1987-03-31'

l5_1987 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [39]:
coleccion1987 = l5_1987.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [40]:
QM_NDVI_1987 = coleccion1987.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1987 = coleccion1987.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1987 = coleccion1987.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1987 = coleccion1987.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1987 = coleccion1987.qualityMosaic('NDSI') # Normalized difference snow index

In [41]:
Map3.addLayer(QM_NDVI_1987, vizParams, 'QM de NDVI 1987')
Map3.addLayer(QM_NDWI_1987, vizParams, 'QM de NDWI 1987')
Map3.addLayer(QM_SAVI_1987, vizParams, 'QM de SAVI 1987')
Map3.addLayer(QM_NDGI_1987, vis_nieve, 'QM de NDGI 1987')
Map3.addLayer(QM_NDSI_1987, vis_nieve, 'QM de NDSI 1987')
Map3

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [42]:
c1987_sn = coleccion1987.map(Cloud20)
NDVI_1987_sn = c1987_sn.qualityMosaic('B11')
NDWI_1987_sn = c1987_sn.qualityMosaic('NDWI')
SAVI_1987_sn = c1987_sn.qualityMosaic('SAVI')
NDGI_1987_sn = c1987_sn.qualityMosaic('NDGI')
NDSI_1987_sn = c1987_sn.qualityMosaic('NDSI')

In [43]:
Map4.addLayer(NDVI_1987_sn, vizParams, 'QMsn de NDVI 1987')
Map4.addLayer(NDWI_1987_sn, vizParams, 'QMsn de NDWI 1987')
Map4.addLayer(SAVI_1987_sn, vizParams, 'QMsn de SAVI 1987')
Map4.addLayer(NDGI_1987_sn, vis_nieve, 'QMsn de NDGI 1987')
Map4.addLayer(NDSI_1987_sn, vis_nieve, 'QMsn de NDSI 1987')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 1988

In [44]:
start_date = '1987-10-01'
end_date = '1988-03-31'

l5_1988 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [45]:
coleccion1988 = l5_1988.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [46]:
QM_NDVI_1988 = coleccion1988.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1988 = coleccion1988.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1988 = coleccion1988.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1988 = coleccion1988.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1988 = coleccion1988.qualityMosaic('NDSI') # Normalized difference snow index

In [47]:
Map3.addLayer(QM_NDVI_1988, vizParams, 'QM de NDVI 1988')
Map3.addLayer(QM_NDWI_1988, vizParams, 'QM de NDWI 1988')
Map3.addLayer(QM_SAVI_1988, vizParams, 'QM de SAVI 1988')
Map3.addLayer(QM_NDGI_1988, vis_nieve, 'QM de NDGI 1988')
Map3.addLayer(QM_NDSI_1988, vis_nieve, 'QM de NDSI 1988')
Map3

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [48]:
c1988_sn = coleccion1988.map(Cloud20)
NDVI_1988_sn = c1988_sn.qualityMosaic('B11')
NDWI_1988_sn = c1988_sn.qualityMosaic('NDWI')
SAVI_1988_sn = c1988_sn.qualityMosaic('SAVI')
NDGI_1988_sn = c1988_sn.qualityMosaic('NDGI')
NDSI_1988_sn = c1988_sn.qualityMosaic('NDSI')

In [49]:
Map4.addLayer(NDVI_1988_sn, vizParams, 'QMsn de NDVI 1988')
Map4.addLayer(NDWI_1988_sn, vizParams, 'QMsn de NDWI 1988')
Map4.addLayer(SAVI_1988_sn, vizParams, 'QMsn de SAVI 1988')
Map4.addLayer(NDGI_1988_sn, vis_nieve, 'QMsn de NDGI 1988')
Map4.addLayer(NDSI_1988_sn, vis_nieve, 'QMsn de NDSI 1988')
# Map4

### 1989

In [50]:
start_date = '1988-10-01'
end_date = '1989-03-31'

l5_1989 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [51]:
coleccion1989 = l5_1989.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [52]:
QM_NDVI_1989 = coleccion1989.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1989 = coleccion1989.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1989 = coleccion1989.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1989 = coleccion1989.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1989 = coleccion1989.qualityMosaic('NDSI') # Normalized difference snow index

In [53]:
Map5.addLayer(QM_NDVI_1989, vizParams, 'QM de NDVI 1989')
Map5.addLayer(QM_NDWI_1989, vizParams, 'QM de NDWI 1989')
Map5.addLayer(QM_SAVI_1989, vizParams, 'QM de SAVI 1989')
Map5.addLayer(QM_NDGI_1989, vis_nieve, 'QM de NDGI 1989')
Map5.addLayer(QM_NDSI_1989, vis_nieve, 'QM de NDSI 1989')
Map5

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [54]:
c1989_sn = coleccion1989.map(Cloud20)
NDVI_1989_sn = c1989_sn.qualityMosaic('B11')
NDWI_1989_sn = c1989_sn.qualityMosaic('NDWI')
SAVI_1989_sn = c1989_sn.qualityMosaic('SAVI')
NDGI_1989_sn = c1989_sn.qualityMosaic('NDGI')
NDSI_1989_sn = c1989_sn.qualityMosaic('NDSI')

In [55]:
Map6.addLayer(NDVI_1989_sn, vizParams, 'QMsn de NDVI 1989')
Map6.addLayer(NDWI_1989_sn, vizParams, 'QMsn de NDWI 1989')
Map6.addLayer(SAVI_1989_sn, vizParams, 'QMsn de SAVI 1989')
Map6.addLayer(NDGI_1989_sn, vis_nieve, 'QMsn de NDGI 1989')
Map6.addLayer(NDSI_1989_sn, vis_nieve, 'QMsn de NDSI 1989')
# Map6

### Colección agua

In [56]:
start_date = '1984-10-01'
end_date = '1989-03-31'

l5_sur = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [57]:
coleccionsur = l5_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [58]:
QM_NDVI_sur = coleccionsur.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_sur = coleccionsur.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_sur = coleccionsur.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_sur = coleccionsur.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_sur = coleccionsur.qualityMosaic('NDSI') # Normalized difference snow index

In [59]:
Map5.addLayer(QM_NDVI_sur, vizParams, 'QM de NDVI sur')
Map5.addLayer(QM_NDWI_sur, vizParams, 'QM de NDWI sur')
Map5.addLayer(QM_SAVI_sur, vizParams, 'QM de SAVI sur')
Map5.addLayer(QM_NDGI_sur, vis_nieve, 'QM de NDGI sur')
Map5.addLayer(QM_NDSI_sur, vis_nieve, 'QM de NDSI sur')
Map5

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [60]:
csur_sn = coleccionsur.map(Cloud20)
NDVI_sur_sn = csur_sn.qualityMosaic('B11')
NDWI_sur_sn = csur_sn.qualityMosaic('NDWI')
SAVI_sur_sn = csur_sn.qualityMosaic('SAVI')
NDGI_sur_sn = csur_sn.qualityMosaic('NDGI')
NDSI_sur_sn = csur_sn.qualityMosaic('NDSI')

In [61]:
Map6.addLayer(NDVI_sur_sn, vizParams, 'QMsn de NDVI sur')
Map6.addLayer(NDWI_sur_sn, vizParams, 'QMsn de NDWI sur')
Map6.addLayer(SAVI_sur_sn, vizParams, 'QMsn de SAVI sur')
Map6.addLayer(NDGI_sur_sn, vis_nieve, 'QMsn de NDGI sur')
Map6.addLayer(NDSI_sur_sn, vis_nieve, 'QMsn de NDSI sur')
# Map6

## Indices por colección

### Índices 1985

In [62]:
NDVI_1985 = NDVI_1985_sn.select('B11')
NDWI_1985 = NDWI_1985_sn.select('NDWI') 
SAVI_1985 = SAVI_1985_sn.select('SAVI')
NDGI_1985 = NDGI_1985_sn.select('NDGI')
NDSI_1985 = NDSI_1985_sn.select('NDSI')

In [63]:
Map7.addLayer(NDVI_1985, {'palette': color_terrain}, 'QMsn de NDVI 1985')
Map7.addLayer(NDWI_1985, {'palette': color_ndwi}, 'QMsn de NDWI 1985')
Map7.addLayer(SAVI_1985, {'palette': color_ndvi}, 'QMsn de SAVI 1985')
# Map7.addLayer(NDGI_1985, {'palette': color_glaciar}, 'QMsn de NDGI 1985')
# Map7.addLayer(NDSI_1985, {'palette': color_ndwi}, 'QMsn de NDSI 1985')
Map7

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 1986

In [64]:
NDVI_1986 = NDVI_1986_sn.select('B11')
NDWI_1986 = NDWI_1986_sn.select('NDWI') 
SAVI_1986 = SAVI_1986_sn.select('SAVI')
NDGI_1986 = NDGI_1986_sn.select('NDGI')
NDSI_1986 = NDSI_1986_sn.select('NDSI')

In [65]:
Map7.addLayer(NDVI_1986, {'palette': color_terrain}, 'QMsn de NDVI 1986')
Map7.addLayer(NDWI_1986, {'palette': color_ndwi}, 'QMsn de NDWI 1986')
Map7.addLayer(SAVI_1986, {'palette': color_ndvi}, 'QMsn de SAVI 1986')
# Map7.addLayer(NDGI_1986, {'palette': color_glaciar}, 'QMsn de NDGI 1986')
# Map7.addLayer(NDSI_1986, {'palette': color_ndwi}, 'QMsn de NDSI 1986')
Map7

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 1987

In [66]:
NDVI_1987 = NDVI_1987_sn.select('B11')
NDWI_1987 = NDWI_1987_sn.select('NDWI') 
SAVI_1987 = SAVI_1987_sn.select('SAVI')
NDGI_1987 = NDGI_1987_sn.select('NDGI')
NDSI_1987 = NDSI_1987_sn.select('NDSI')

In [67]:
# Map8.addLayer(NDVI_1987, {'palette': color_terrain}, 'QMsn de NDVI 1987')
# Map8.addLayer(NDWI_1987, {'palette': color_ndwi}, 'QMsn de NDWI 1987')
# Map8.addLayer(SAVI_1987, {'palette': color_ndvi}, 'QMsn de SAVI 1987')
# Map8.addLayer(NDGI_1987, {'palette': color_glaciar}, 'QMsn de NDGI 1987')
# Map8.addLayer(NDSI_1987, {'palette': color_ndwi}, 'QMsn de NDSI 1987')
# Map8

### Índices 1988

In [68]:
NDVI_1988 = NDVI_1988_sn.select('B11')
NDWI_1988 = NDWI_1988_sn.select('NDWI') 
SAVI_1988 = SAVI_1988_sn.select('SAVI')
NDGI_1988 = NDGI_1988_sn.select('NDGI')
NDSI_1988 = NDSI_1988_sn.select('NDSI')

In [69]:
# Map8.addLayer(NDVI_1988, {'palette': color_terrain}, 'QMsn de NDVI 1988')
# Map8.addLayer(NDWI_1988, {'palette': color_ndwi}, 'QMsn de NDWI 1988')
# Map8.addLayer(SAVI_1988, {'palette': color_ndvi}, 'QMsn de SAVI 1988')
# Map8.addLayer(NDGI_1988, {'palette': color_glaciar}, 'QMsn de NDGI 1988')
# Map8.addLayer(NDSI_1988, {'palette': color_ndwi}, 'QMsn de NDSI 1988')
# Map8

### Índices 1989

In [70]:
NDVI_1989 = NDVI_1989_sn.select('B11')
NDWI_1989 = NDWI_1989_sn.select('NDWI') 
SAVI_1989 = SAVI_1989_sn.select('SAVI')
NDGI_1989 = NDGI_1989_sn.select('NDGI')
NDSI_1989 = NDSI_1989_sn.select('NDSI')

In [71]:
# Map8.addLayer(NDVI_1989, {'palette': color_terrain}, 'QMsn de NDVI 1989')
# Map8.addLayer(NDWI_1989, {'palette': color_ndwi}, 'QMsn de NDWI 1989')
# Map8.addLayer(SAVI_1989, {'palette': color_ndvi}, 'QMsn de SAVI 1989')
# Map8.addLayer(NDGI_1989, {'palette': color_glaciar}, 'QMsn de NDGI 1989')
# Map8.addLayer(NDSI_1989, {'palette': color_ndwi}, 'QMsn de NDSI 1989')
# Map8

### Índices colección agua

In [149]:
NDVI_sur = NDVI_sur_sn.select('B11')
NDWI_sur = NDWI_sur_sn.select('NDWI') 
SAVI_sur = SAVI_sur_sn.select('SAVI')
NDGI_sur = NDGI_sur_sn.select('NDGI')
NDSI_sur = NDSI_sur_sn.select('NDSI')

In [150]:
Map8.addLayer(NDVI_sur, {'palette': color_terrain}, 'QMsn de NDVI sur')
Map8.addLayer(NDWI_sur, {'palette': color_ndwi}, 'QMsn de NDWI sur')
Map8.addLayer(SAVI_sur, {'palette': color_ndvi}, 'QMsn de SAVI sur')
Map8.addLayer(NDGI_sur, {'palette': color_glaciar}, 'QMsn de NDGI sur')
Map8.addLayer(NDSI_sur, {'palette': color_ndwi}, 'QMsn de NDSI sur')
Map8

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Generación de Mosaico

In [110]:
mosaic0= ee.ImageCollection([QM_NDVI_sur,QM_NDVI_1987]).mosaic()

In [111]:
mosaic1 = ee.ImageCollection([mosaic0,QM_NDVI_1985]).mosaic()

In [112]:
mosaic2 = ee.ImageCollection([mosaic1,QM_NDVI_1986]).mosaic()

In [113]:
mosaic2_landsat = mosaic2.set('SENSOR_ID', 'OLI_TIRS')

In [114]:
cloud_mosaic2 = ee.Algorithms.Landsat.simpleCloudScore(mosaic2_landsat)

In [115]:
cloud_mosaic2 = cloud_mosaic2.select('cloud')

In [116]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [117]:
resultados= [cloud_mosaic2, mosaic2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes 1985',
    'QM NDVI 1985']

In [118]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [119]:
def llenado(image):
    return image.where(cloud_mosaic2.gt(20),NDVI_1985_sn)

In [120]:
img1 = llenado(mosaic2)

In [121]:
img1_landsat = img1.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [122]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [123]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [124]:
def llenado2(image):
    return image.where(cloud_img1.gt(20), NDVI_1986_sn)

In [125]:
img2 = llenado2(img1)

In [126]:
img2_landsat = img2.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [127]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [128]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [129]:
def llenado3(image):
    return image.where(cloud_img2.gt(20), NDVI_1987_sn)

In [130]:
img3 = llenado3(img2)

In [131]:
img3_landsat = img3.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [132]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [133]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [134]:
def llenado4(image):
    return image.where(cloud_img3.gt(20), NDVI_1988_sn)

In [135]:
img4 = llenado4(img3)

In [136]:
img4_landsat = img4.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [137]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [138]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [139]:
def llenado5(image):
    return image.where(cloud_img4.gt(20), NDVI_1989_sn)

In [140]:
img5 = llenado5(img4)

In [141]:
img5_landsat = img5.set('SENSOR_ID', 'OLI_TIRS')
cloud_img5 = ee.Algorithms.Landsat.simpleCloudScore(img5_landsat)
cloud_img5 = cloud_img5.select('cloud')

In [142]:
resultados= [cloud_img5, img5]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 5',
    'Mosaico 5']

In [143]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [144]:
def llenado6(image):
    return image.where(NDVI_sur_sn.lt(0.25).And(NDWI_sur_sn.gt(0.3)), NDWI_sur_sn)

In [145]:
img6 = llenado6(img5)

In [146]:
resultados= [img5, img6]
labels = [
    'Mosaico 5',
    'Mosaico 6']
vis_params3=  [
     vizParams, 
    vizParams]

In [147]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [148]:
Map10.addLayer(img5, vizParams, 'Mosaico 5')
Map10.addLayer(img6, vizParams, 'Mosaico 6')
Map10

Map(bottom=5532.0, center=[-44.59046718130884, -61.12354727110302], controls=(WidgetControl(options=['position…